In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
import envelope_plot as ep
from ipywidgets import interact
import ipywidgets as widgets
import os

cf.go_offline()
plotly.offline.init_notebook_mode(connected=True)

def extract_keys(my_dict, key_list):
    return {k: my_dict[k] for k in key_list}
    

# Prepare data

Every domain instance requires the optimal path length which is
read in via a json results file in the same format as the experiment results.

Specify the paths to the results yielding information on the optimal plans below.
Then specify paths to the experiment results file. A dataframe will be constructed
using this information


In [ ]:
optimal_plans = ['../results/astar_minima1500.json','../results/astar_orz100d.json',
                 '../results/astar_korf4real.json','../results/astar_hansen-bigger-d-wide3_no-startup.json']
experiments = [
    '../results/dataRACETRACK_BI_LSS-09-19-02-07-19.json',
    '../results/dataRACETRACK_TBA-14-05-13-07-19.json',
    '../results/dataLIMITED.json',
    '../results/test.json'
]

experiment_data = ep.analyze_results(optimal_plans, experiments)

# Plot data

Please use the drop down menus to select the domain instance and the plot type

In [ ]:
domains = {
    'Minima 1500x1500':'minima1500',
    'Dragon Age: Origins':'orz100d',
    'Sliding Tile - Korf 4x4':'tiles/korf/4/real',
    'Racetrack (Hansen, 3-wide)': 'hansen-bigger-d-wide3'
}

master_config = {
    'weight': 'W',
    'lookaheadStrategy': 'Look',
    'envelopeSearchStrategy': 'Env-L',
    'frontierAdjustmentRatio': 'Adj'
}

just_weight = extract_keys(master_config, ['weight'])
wLookaheadConfig = extract_keys(master_config, ['weight', 'lookaheadStrategy'])
biEsConfig = extract_keys(master_config, [
    'weight', 'lookaheadStrategy', 'envelopeSearchStrategy', 'frontierAdjustmentRatio'
])

groupings = {
    'LSS_LRTA_STAR':{},
    'BI_ES': biEsConfig,
    'BACK_ES': just_weight,
    'TIME_BOUNDED_A_STAR': wLookaheadConfig
}

within_opt_data = ep.prepare_for_within_opt_plot(experiment_data,domains.values(), groupings)

@interact(
    domain=domains,
    display_type=['Plot','Table'],
    algorithms=widgets.SelectMultiple(
        options={
            'TBA*':'TIME_BOUNDED_A_STAR',
            'LSS-LRTA*':'LSS_LRTA_STAR',
            'Bidirectional Envelope': 'BI_ES',
            'Backward Envelope': 'BACK_ES'
        },
        description='Algorithms (Select multiple)',
        value=[]
    ),
    weights=widgets.SelectMultiple(
        options=list(filter(lambda x: x==x, experiment_data.weight.unique())), # remove NaN
        description='Weights (Select multiple, or none for all)',
        value=[]
    ),
    bi_lookahead=widgets.SelectMultiple(
        options={
            'Greedy Env, Greedy Frontier':0,
            'Greedy Env, A* Frontier': 1,
            'A* Env, Greedy Frontier': 2,
            'A* Env, A* Frontier': 3
        },
        description='Bi-ES lookahead strategies',
        value=[2]
    ),
    adjustmentRatios=widgets.SelectMultiple(
        options=list(filter(lambda x: x==x, experiment_data.frontierAdjustmentRatio.unique())), # remove NaN
        description='Frontier Adjustment Ratio (BI-ES)',
        value=[]
    )
)
def plot_within_opt(domain, display_type, algorithms, weights, bi_lookahead, adjustmentRatios):
    data_copy = within_opt_data[domain].copy() # for safety - don't modify original
    
    if len(algorithms) == 0:
        return
    
    trimmed_df = pd.DataFrame({}, columns=data_copy.columns)
    
    for alg in algorithms:
        alg_data = data_copy[data_copy.algCode == alg]
        
        # Filters
    
        # Generic filter loop - if param exists and values are specified, filter by it
    
        alg_filters = groupings[alg]
        for group, values in zip(
            ['weight', 'frontierAdjustmentRatio'],
            [weights, adjustmentRatios]
        ):
            if group in alg_filters and len(values) > 0:
                alg_data = alg_data[alg_data[group].isnull() | alg_data[group].isin(values)]

            
        # Special Bi-ES: filter by lookahead strategy
        if alg == 'BI_ES':
            if 0 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 1 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'GBFS'))]
            if 2 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'GBFS') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            if 3 not in bi_lookahead:
                alg_data = alg_data[~((alg_data.lookaheadStrategy == 'A_STAR') &
                                   (alg_data.envelopeSearchStrategy == 'A_STAR'))]
            
        
        trimmed_df = trimmed_df.append(alg_data)
        
        
    if display_type == 'Table':
        table = ff.create_table(trimmed_df)
        table.layout.update({'title':domain})
        plotly.iplot(table, filename='jupyter-table')
    elif display_type == 'Plot':
        # cufflinks doesn't support error bars...? At least I can't find it
        # creating plotly plot by hand instead... ugh!
        
        palette = cf.colors.get_scales('spectral', trimmed_df.algorithmName.nunique())
        num_colors = len(palette)
        data_comp = []
        for idx, alg_name in enumerate(trimmed_df.algorithmName.unique()):
            alg_df = trimmed_df[trimmed_df.algorithmName == alg_name]
            data_comp.append(go.Scatter(
                x = alg_df.actionDuration,
                y = alg_df.withinOpt,
                name = alg_name,
                mode = 'lines+markers',
                error_y = dict(
                    type='data',
                    symmetric=False,
                    array=alg_df.lbound,
                    arrayminus=alg_df.rbound,
                    visible=True
                ),
                line = dict(
                    color=palette[idx % num_colors]
                )
            ))
        
        layout_comp = go.Layout(
            title=domain,
            xaxis=dict(
                title='Action Duration',
                type='log'
            ),
            yaxis=dict(
                title='Within Optimal'
            )
        )
        
        fig_comp = go.Figure(data=data_comp, layout=layout_comp)
        
        plotly.iplot(fig_comp, filename='jupyter-plot')
        

In [ ]:
# test = experiment_data[(experiment_data['algorithmName'] == 'BI_ES') &
#                        (experiment_data['domainName'] == 'RACETRACK') &
#                        (experiment_data['actionDuration'] == 10)
#                       ]

# resetCount = 0
# index = 0

# for idx, row in test.iterrows():
#     if 'envelopeResets' in row.attributes:
#         resetCount += row.attributes['envelopeResets']
#         index += 1
    
# avg = resetCount / index
# print(avg)

print(cf.colors.get_scales('spectral', 5))